In [4]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [3]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\MUNIC")

In [6]:
df18=pd.read_csv("MUNIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("MUNIC_educacao_2021.csv", sep=';')

In [13]:
df21['raca_cor']

0        Parda
1        Parda
2        Parda
3        Parda
4        Parda
         ...  
5565     Parda
5566    Branca
5567     Parda
5568     Parda
5569    Branca
Name: raca_cor, Length: 5570, dtype: object

In [14]:
df18 = df18[['Cod Municipio', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
df21 = df21[['CodMun', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
        

In [15]:
df18['ano']=2018
df21['ano']=2021

In [18]:
df18 = df18.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio'})
df21 = df21.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio'})

In [22]:
df18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_municipio  5570 non-null   int64 
 1   genero        5570 non-null   object
 2   idade         5570 non-null   object
 3   raca_cor      5570 non-null   object
 4   escolaridade  5570 non-null   object
 5   ano           5570 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 261.2+ KB


In [20]:
df21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_municipio  5570 non-null   int64 
 1   genero        5570 non-null   object
 2   idade         5570 non-null   object
 3   raca_cor      5570 non-null   object
 4   escolaridade  5570 non-null   object
 5   ano           5570 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 261.2+ KB


In [23]:
df = pd.concat([df18,df21])

In [35]:
df['idade'].unique()

array(['61', '45', '59', '54', '40', '35', '57', '42', '41', '66', '30',
       '51', '49', '28', '32', '36', '63', '44', '46', '33', '38', '43',
       '39', '52', '53', '60', '37', '48', '64', '26', '29', '47', '31',
       '50', '58', '55', '34', '56', '62', nan, '77', '68', '69', '79',
       '71', '65', '67', '25', '21', '75', '70', '27', '23', '74', '76',
       '72', '78', '73', '19', '24', '22', '80', 'Não informou', '81'],
      dtype=object)

In [36]:
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade'] =pd.to_numeric(df['idade'])


In [37]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [40]:
x = df.pivot_table(columns=['ano','genero', 'faixa_etaria', 'raca_cor',
        'escolaridade'], aggfunc='size')
y = pd.DataFrame(x)
y = y.reset_index() ##inserir a contagem como coluna
y = y.rename(columns={0:'count'})

In [41]:
df = y[y['count']>0]

In [45]:
sum(df[df['ano']==2021]['count'])

5558

In [ ]:
### Subindo para o GBQ

In [5]:
client = bigquery.Client()

In [13]:
dataset_ref = client.dataset('cargos_lideranca')

In [50]:
df=df.rename(columns={'count':'quantidade','raca_cor':'cor_raca'})

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 14 to 1831
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ano           287 non-null    int64   
 1   genero        287 non-null    object  
 2   faixa_etaria  287 non-null    category
 3   cor_raca      287 non-null    object  
 4   escolaridade  287 non-null    object  
 5   quantidade    287 non-null    int64   
dtypes: category(1), int64(2), object(3)
memory usage: 13.9+ KB


In [11]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('escolaridade','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
 bigquery.SchemaField('quantidade','INTEGER',description='Área deformação.')
 ]

In [14]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_educacao_consolidado')

In [15]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [16]:
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=ec8a17fa-2646-4597-b46e-b1ed18d6a258>

In [ ]:
## Fazendo alterações. 

In [1]:
query = "select * from `repositoriodedadosgpsp.cargos_lideranca.MUNIC_perfil_gestor_educacao_consolidado`" 

In [6]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 287/287 [00:00<00:00, 657.37rows/s]


In [10]:
df['escolaridade'].unique()

array(['Até Ensino Médio', 'Ensino superior completo',
       'Ensino superior incompleto', 'Especialização', 'Mestrado',
       'Até Ensino Fundamental', 'Doutorado'], dtype=object)

In [9]:
df['escolaridade'] = np.where(df['escolaridade']=="Ensino médio (2º Grau) completo","Até Ensino Médio",df['escolaridade'])
df['escolaridade'] = np.where(df['escolaridade']=="Ensino superior completo","Ensino superior completo",df['escolaridade'])
df['escolaridade'] = np.where(df['escolaridade']=="Ensino superior incompleto","Ensino superior incompleto",df['escolaridade'])
df['escolaridade'] = np.where(df['escolaridade']=="Ensino fundamental (1º Grau) completo","Até Ensino Fundamental",df['escolaridade'])
df['escolaridade'] = np.where(df['escolaridade']=="Ensino médio (2º Grau) incompleto","Até Ensino Médio",df['escolaridade'])
df['escolaridade'] = np.where(df['escolaridade']=="Ensino fundamental (1º Grau) incompleto","Até Ensino Fundamental",df['escolaridade'])